
# 💸 Compound Interest + Frugality Lab (Generic Starter)

Don’t know your future job yet? This **generic** notebook lets you start with a single
**median starting salary** (default provided, easily changed) and see how
**consistent investing + small frugality tweaks** compound over ~30 years.

- **Assume best-case investment returns** by default (you can adjust).  
- **Ignore inflation** so the power of compounding is front-and-center.  
- Toggle common **spending leaks** and redirect part/all of them into investing.



## How to use
1) Run **Setup**.  
2) In **Inputs**, set your **starting salary** (or keep the default) and choose your frugality toggles.  
3) Run **Simulate** to compare **Base** vs **Frugality** scenarios with year-by-year tables and charts.


## 0) Setup

In [ ]:

import math
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, Any

pd.set_option("display.max_colwidth", None)

def link(text, url):
    return f'<a href="{url}" target="_blank" rel="noopener noreferrer">{text}</a>'

GENERIC_PRESET = {
    "name": "Undecided / Generic Starter",
    # Default median starting salary across professions (editable by student)
    # Use this as a placeholder; replace with your local or researched number.
    "starting_salary": 45000,
    "source_text": "Choose a starting salary (see BLS OOH/OEWS to research)",
    "source_link_1_text": "BLS Occupational Outlook Handbook (A–Z)",
    "source_link_1_url": "https://www.bls.gov/ooh/a-z-index.htm",
    "source_link_2_text": "BLS OEWS (occupation pay data)",
    "source_link_2_url": "https://www.bls.gov/oes/",
}

CADENCE_PERIODS = {
    "Weekly (52/yr)": 52,
    "Bi-Weekly (26/yr)": 26,
    "Monthly (12/yr)": 12,
    "Quarterly (4/yr)": 4,
    "Yearly (1/yr)": 1,
}

def annual_to_per_period(annual_amount: float, periods_per_year: int) -> float:
    return annual_amount / periods_per_year

def fmt_currency(x: float) -> str:
    return f"${x:,.0f}"

print("✔️ Setup complete.")



## 1) Inputs (edit and run)

- **Starting salary**: a single number for now; you can add raises in the assumption below.  
- **Contribution % of income** and **Cadence**: weekly/bi-weekly/monthly/quarterly/yearly.  
- **Expected return (nominal)**: default **10%** (best-case for demo); adjust if desired.  
- **Inflation**: **ignored** in this starter notebook.  
- **Spending leaks**: choose categories and redirection fractions.


In [ ]:

# ---- Main knobs ----
YEARS = 30
CADENCE_LABEL = "Monthly (12/yr)"
BASE_CONTRIB_RATE = 0.10        # 10% of gross income
EXPECTED_RETURN = 0.10           # best-case default; adjust as you like
ANNUAL_RAISE = 0.025             # simple yearly raise assumption
FEES = 0.00                      # keep at 0% to spotlight compounding
INITIAL_BALANCE = 0.0
ONE_TIME_LUMP_SUM = 0.0

# Use the generic default, or overwrite this value with your own research:
STARTING_SALARY = GENERIC_PRESET["starting_salary"]

# ---- Frugality "spending leaks" ----
LEAKS = {
    "Coffee shop drink":     {"price": 5.00,  "freq": 5, "unit": "week",  "redirect": 1.00},
    "Dining out (casual)":   {"price": 15.00, "freq": 2, "unit": "week",  "redirect": 0.50},
    "Energy drink":          {"price": 3.00,  "freq": 5, "unit": "week",  "redirect": 1.00},
    "Streaming bundle":      {"price": 25.00, "freq": 1, "unit": "month", "redirect": 1.00},
    "Ride-hailing trips":    {"price": 20.00, "freq": 1, "unit": "week",  "redirect": 0.50},
    "In-app purchases":      {"price": 10.00, "freq": 1, "unit": "week",  "redirect": 0.75},
}

def leaks_to_annual(leaks: Dict[str, Dict[str, Any]]) -> pd.DataFrame:
    rows = []
    for name, cfg in leaks.items():
        per = cfg["price"] * cfg["freq"]
        annual = per * (52 if cfg["unit"] == "week" else 12)
        redir = annual * float(cfg.get("redirect", 1.0))
        rows.append({
            "Category": name,
            "Unit Price": cfg["price"],
            "Frequency": cfg["freq"],
            "Unit": cfg["unit"],
            "Annual Spend": annual,
            "Redirect Fraction": cfg.get("redirect", 1.0),
            "Annual Redirect to Investing": redir,
        })
    return pd.DataFrame(rows)

LEAKS_DF = leaks_to_annual(LEAKS)
LEAKS_DF_DISPLAY = LEAKS_DF.copy()
for c in ["Unit Price", "Annual Spend", "Annual Redirect to Investing"]:
    LEAKS_DF_DISPLAY[c] = LEAKS_DF_DISPLAY[c].map(fmt_currency)
LEAKS_DF_DISPLAY


## 2) Simulator

In [ ]:

from dataclasses import dataclass

@dataclass
class Assumptions:
    years: int
    starting_salary: float
    annual_raise_pct: float
    contrib_rate_of_income: float
    cadence_label: str
    expected_return: float
    annual_fee_drag: float
    initial_balance: float
    one_time_lump_sum: float
    annual_redirect_extra: float

def simulate(a: Assumptions, title: str):
    periods = CADENCE_PERIODS[a.cadence_label]
    dt = 1/periods

    rows = []
    balance = a.initial_balance + a.one_time_lump_sum
    salary = a.starting_salary

    for year in range(a.years + 1):
        annual_contrib_employee = salary * a.contrib_rate_of_income
        annual_contrib_total = annual_contrib_employee + a.annual_redirect_extra
        per_period_employee = annual_to_per_period(annual_contrib_total, periods)

        period_rate_net = (1 + a.expected_return - a.annual_fee_drag) ** dt - 1

        year_contrib = 0.0
        for _ in range(periods):
            balance = balance * (1 + period_rate_net) + per_period_employee
            year_contrib += per_period_employee

        rows.append({
            "Year": year,
            "Salary (annual)": salary,
            "Total Contribution (annual)": year_contrib,
            "End Balance": balance,
        })

        salary *= (1 + a.annual_raise_pct)

    import pandas as pd
    df = pd.DataFrame(rows)
    df.attrs["title"] = title
    return df

def df_money_format(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in ["Salary (annual)", "Total Contribution (annual)", "End Balance"]:
        out[c] = out[c].map(fmt_currency)
    return out


## 3) Run: Base vs. Frugality

In [ ]:

annual_redirect = float(LEAKS_DF["Annual Redirect to Investing"].sum())

base = Assumptions(
    years=YEARS, starting_salary=STARTING_SALARY,
    annual_raise_pct=ANNUAL_RAISE, contrib_rate_of_income=BASE_CONTRIB_RATE,
    cadence_label=CADENCE_LABEL, expected_return=EXPECTED_RETURN,
    annual_fee_drag=FEES, initial_balance=INITIAL_BALANCE,
    one_time_lump_sum=ONE_TIME_LUMP_SUM, annual_redirect_extra=0.0
)

frugal = Assumptions(**{**base.__dict__, "annual_redirect_extra": annual_redirect})

df_base = simulate(base, title="Base")
df_frugal = simulate(frugal, title="Frugality")

final_base = df_base["End Balance"].iloc[-1]
final_frugal = df_frugal["End Balance"].iloc[-1]
delta = final_frugal - final_base

from IPython.display import HTML, display
import pandas as pd
summary = pd.DataFrame([
    {"Scenario": "Base (income % only)", "Final Balance": fmt_currency(final_base), "Annual Redirect Added": fmt_currency(0.0)},
    {"Scenario": "Frugality (base + redirected leaks)", "Final Balance": fmt_currency(final_frugal), "Annual Redirect Added": fmt_currency(annual_redirect)},
])

display(HTML(f"""
<h4>Generic Starter</h4>
<p>Starting Salary (editable): <b>{fmt_currency(STARTING_SALARY)}</b><br/>
Sources to research your own starting salary:
&nbsp;• {link(GENERIC_PRESET['source_link_1_text'], GENERIC_PRESET['source_link_1_url'])}
&nbsp;• {link(GENERIC_PRESET['source_link_2_text'], GENERIC_PRESET['source_link_2_url'])}
</p>
"""))
display(summary)

print("Annual redirect to investing from selected leaks:", fmt_currency(annual_redirect))

print("\n— Base plan (year-by-year) —")
display(df_money_format(df_base))

print("\n— Frugality plan (year-by-year) —")
display(df_money_format(df_frugal))

plt.figure(figsize=(10,5))
plt.plot(df_base["Year"], df_base["End Balance"], label="Base")
plt.plot(df_frugal["Year"], df_frugal["End Balance"], label="Frugality")
plt.title("End Balance Over Time — Base vs. Frugality")
plt.xlabel("Year")
plt.ylabel("Balance")
plt.legend()
plt.show()

contrib_cum = df_frugal["Total Contribution (annual)"].cumsum()
plt.figure(figsize=(10,5))
plt.plot(df_frugal["Year"], contrib_cum, label="Total Contributions (cum.)")
plt.plot(df_frugal["Year"], df_frugal["End Balance"], label="End Balance (Frugality)")
plt.title("Frugality: Contributions vs. Compounded Balance")
plt.xlabel("Year")
plt.ylabel("Dollars")
plt.legend()
plt.show()

print(f"""
Key takeaways:
- Redirected spending adds **{fmt_currency(annual_redirect)} per year** to investing.
- Over {YEARS} years at {EXPECTED_RETURN*100:.1f}% nominal with {CADENCE_LABEL.lower()}, that difference compounds to **{fmt_currency(delta)}** more than the base plan.
- Replace STARTING_SALARY with your own researched number for a personalized run.
""" )



## 4) Quick Links to Research Your Own Starting Salary
- BLS Occupational Outlook Handbook (A–Z): https://www.bls.gov/ooh/a-z-index.htm  
- BLS OEWS (occupation pay data): https://www.bls.gov/oes/  
